## Cell 1: Import thư viện

In [1]:
import pandas as pd
from collections import Counter
from pyvi import ViTokenizer
from tqdm import tqdm
import sys
import os

## Cell 2: Xử lý Blank và Lọc dữ liệu (Quan trọng)

In [2]:
# Đường dẫn tương đối từ thư mục notebooks/
input_path = '../data/raw/dataset_raw.csv'
clean_path = '../data/raw/dataset_filtered.csv'

print("1. Đang lọc dữ liệu thô...")
df = pd.read_csv(input_path)

# Logic xóa dòng Null Label, cmt rỗng
df_clean = df.dropna(subset=['label', 'comment_text'])
df_clean = df_clean[df_clean['comment_text'].astype(str).str.strip() != '']
df_clean.to_csv(clean_path, index=False, encoding='utf-8-sig')

print(f"Đã lưu file sạch tại: {clean_path}")

1. Đang lọc dữ liệu thô...
Đã lưu file sạch tại: ../data/raw/dataset_filtered.csv


## Cell 3: Khảo sát độ phủ (Step 1)
Mục đích: Chạy đoạn này để nhìn bảng kết quả, xem ở Top bao nhiêu thì độ phủ đạt >70% (Điểm bão hòa).

In [3]:
# 1. Đọc và Tách từ
print("Đang đọc dữ liệu và tách từ...")
df = pd.read_csv(clean_path)
# Lấy cột text (sửa tên cột nếu khác)
texts = df['comment_text'].dropna().astype(str).tolist()

all_words = []
for text in tqdm(texts, desc="Tokenizing"):
    # Clean nhẹ + Tách từ PyVi
    words = ViTokenizer.tokenize(text.lower()).split()
    all_words.extend(words)

# 2. Tính toán độ phủ
word_counts = Counter(all_words)
total_occurrences = len(all_words)
sorted_words = word_counts.most_common()

current_sum = 0
checkpoints = [100, 200, 500, 1000, 2000] # Các mốc muốn kiểm tra

print(f"\n{'Top N từ':<10} | {'Độ phủ (%)':<15}")
print("-" * 30)

for i, (word, count) in enumerate(sorted_words):
    current_sum += count
    rank = i + 1
    if rank in checkpoints:
        percent = (current_sum / total_occurrences) * 100
        print(f"{rank:<10} | {percent:.2f}%")

Đang đọc dữ liệu và tách từ...


Tokenizing: 100%|██████████| 6124/6124 [00:00<00:00, 24430.62it/s]


Top N từ   | Độ phủ (%)     
------------------------------
100        | 44.99%
200        | 57.43%
500        | 72.60%
1000       | 82.17%
2000       | 90.04%


## Cell 4: 
**Nhận xét**: Dựa trên bảng thống kê trên, nhóm nhận thấy tại ngưỡng Top 500, độ phủ từ vựng đã đạt mức bão hòa (~72%). Việc mở rộng thêm không mang lại hiệu quả đáng kể so với chi phí xử lý. 

=> **Quyết định**: Chọn ngưỡng cắt Top 500 để trích xuất ứng viên Teencode và Stopwords

## Cell 5: 
Xuất file

In [4]:
TOP_N = 500  # Ngưỡng cắt cho từ điển
OUTPUT_FOLDER = '../data/dictionaries/'
if not os.path.exists(OUTPUT_FOLDER): os.makedirs(OUTPUT_FOLDER)

print(f"\nĐang xuất file với ngưỡng cắt: Top {TOP_N}...")

# 1. Xuất file STOPWORDS (Lấy thẳng Top N từ xuất hiện nhiều nhất)
# Lý thuyết Zipf: Những từ xuất hiện nhiều nhất thường là hư từ
df_stop = pd.DataFrame(sorted_words[:TOP_N], columns=['Word', 'Frequency'])
df_stop.to_csv(os.path.join(OUTPUT_FOLDER, 'candidates_stopwords.csv'), index=False, encoding='utf-8-sig')
print("Đã xuất: candidates_stopwords.csv")

# 2. Xuất file TEENCODE (Lọc từ danh sách từ vựng, ưu tiên từ phổ biến)
vowels = set("aáàảãạăắằẳẵặâấầẩẫậeéèẻẽẹêếềểễệiíìỉĩịoóòỏõọôốồổỗộơớờởỡợuúùủũụưứừửữựyýỳỷỹỵ")
potential_teencode = []

# Chỉ quét trong những từ đã xuất hiện (sorted_words)
for word, freq in sorted_words:
    if '_' in word: continue # Bỏ qua từ ghép chuẩn (học_sinh)
    
    # Luật 1: Từ ngắn (<3 ký tự) xuất hiện nhiều (k, ko, dc...)
    rule_short = (len(word) < 3 and freq > 10)
    # Luật 2: Từ không có nguyên âm (tr, bt, cmn...)
    rule_no_vowel = (not any(c in vowels for c in word) and word.isalpha())
    # Luật 3: Từ chứa số (k4, q3...)
    rule_has_num = (any(c.isdigit() for c in word) and not word.isdigit())

    if rule_short or rule_no_vowel or rule_has_num:
        potential_teencode.append((word, freq))

# Cắt lấy đúng Top N teencode 
df_teen = pd.DataFrame(potential_teencode, columns=['Word', 'Frequency'])
df_teen = df_teen.head(TOP_N) 
df_teen.to_csv(os.path.join(OUTPUT_FOLDER, 'candidates_teencode.csv'), index=False, encoding='utf-8-sig')
print("Đã xuất: candidates_teencode.csv")


Đang xuất file với ngưỡng cắt: Top 500...
Đã xuất: candidates_stopwords.csv
Đã xuất: candidates_teencode.csv


## Cell 6: Step 2 – Rule-based Mining (Teencode ẩn)

**Vấn đề**: Phương pháp thống kê tần suất (Step 1) có thể bỏ sót các teencode "ẩn" chứa ký tự đặc biệt hoặc số, ví dụ: `chao2`, `haizzz`.

**Giải pháp**: Nhóm đã sử dụng công cụ **Rule-based Mining** được đóng gói trong `src/step2_rule_based_mining.py` để quét lại dữ liệu và trích xuất các từ teencode tiềm năng.  
- Tool này chạy độc lập từ terminal/IDE, đọc dữ liệu sạch `dataset_filtered.csv` và xuất kết quả `teencode_candidates_step2.csv`.
- Trong notebook, chỉ ghi nhận bước này mà không cần gọi code.
